# Chapter 4: 
# Cải thiện khả năng dự báo của mô hình bằng Feature Engineering

Như đã đề cập trong chương 1, Feature Engineering (FE) được hiểu như là việc biến đổi - chuyển hóa dữ liệu nguyên bản (Original Feature) thành Feature mới sao cho khả năng dự báo và phân loại của mô hình ML tăng lên. FE có thể là kĩ thuật đơn giản chuyển hóa biến định tính (Categorical) về biến Binary 0 - 1 vì rằng hầu hết các mô hình ML được xây dựng trong Python không chấp nhận Feature là biến định tính. FE không chỉ là biến đổi - chuyển hóa các features ban đầu thành các features mới mà còn có thể là việc tìm ra một feature mới (bằng một sự kết hợp - biến đổi nào đó từ các features đã có) để nâng cao khả năng phân loại - dựa báo của mô hình. 

Chương này hướng vào các nội dung sau: 

- Giới thiệu về FE cho các mô hình ML. Để minh họa thì kĩ thuật chuẩn hóa 0 - 1 được sử dụng với mô hình KNN. 
- Tinh chỉnh tham số kết hợp với FE và Pipelines. 
- So sánh chất lượng dự báo - phân loại của mô hình khi không và có sử dụng FE. 

# 4.1 Feature Engineering 

Trong chương 1 chúng ta đã thấy với K = 1 và sử dụng dữ liệu nguyên bản thì Accuracy = 57.6%. Mức độ chính xác khi phân loại như vậy là không đủ tốt để sử dụng theo các tiêu chí đánh giá như đã trình bày trong chương 3. Để nâng cao khả năng phân loại - dự báo của mô hình chúng ta có thể tìm tham số tối ưu thông qua quá trình tinh chỉnh tham số đã được trình bày trong chương 2. 

Tuy nhiên với FE chúng ta có thể cái thiện chất lượng dự báo - phân loại của mô hình một cách nhanh chóng. Để minh họa chúng ta trở lại với mô hình KNN với K = 1 và so sánh Accuracy trong hai tình huống: sử dụng và không sử dụng FE khi huấn luyện mô hình. Bằng kĩ thuật chuẩn hóa 0 - 1 đơn giản chúng ta có thể tăng Accuracy lên 63.55%. Trước hết thực hiện chuẩn bị dữ liệu: 

In [1]:
#========================================
#  Chuẩn bị dữ liệu cho huấn luyện KNN
#========================================

import pandas as pd
df_bank = pd.read_csv("C:/Users/Zbook/Desktop/DataMining/dmba/UniversalBank.csv")
my_df_binary = df_bank.drop(["ZIP Code", "ID"], axis=1)
Y = my_df_binary["CreditCard"]
X = my_df_binary.drop("CreditCard", 1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.4, random_state = 29, stratify = Y)

In [2]:
# Huấn luyện KNN với K = 1: 
from sklearn.neighbors import KNeighborsClassifier
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(X_train, y_train)

# Tính Accuracy và hiển thị kết quả: 
accuracy_noneScaled = knn1.score(X_test, y_test)
print("Accuracy nếu sử dụng dữ liệu nguyên bản:", accuracy_noneScaled)

Accuracy nếu sử dụng dữ liệu nguyên bản: 0.576


KNN là một thuật toán dựa vào khoảng cách và do vậy rất nhạy với thước đo được sử dụng cho Features. Kinh nghiệm được đo theo đơn vị là năm và nó là một con số khá bé (nằm đâu đó từ 0 đến 50 là cùng) so với thu nhập có đơn vị, ví dụ, là VND và con số thu nhập có thể từ vài ba triệu cho đến hàng trăm triệu. Do vậy chúng ta cần chuẩn hóa tất cả các thước đo này về *cùng một thước đo*. 

Bằng sử dụng chuẩn hóa 0 - 1 cho các features chúng ta có thể cải thiện đáng kể chất lượng của mô hình. Chú ý rằng khi huấn luyện thì chúng ta phải huấn luyện trên dữ liệu train đã được chuẩn hóa còn dự báo chúng ta phải thực hiện trên dữ liệu test. Scikit-Learn hỗ trợ nhiều kĩ thuật FE cho features chứ không chỉ kĩ thuật chuẩn hóa 0 - 1. Dưới đây là Python Codes thực hiện kĩ thuật chuẩn hóa này: 

In [3]:
# Load hàm MinMaxScaler() từ Sciki-Learn cho chuẩn hóa 0 - 1: 
from sklearn.preprocessing import MinMaxScaler

# Kích hoạt hàm chuẩn hóa: 
scaler = MinMaxScaler().fit(X_train)

# Chuẩn hóa 0 - 1 cho train data: 
X_train_scaled = scaler.transform(X_train)

# Huấn luyện KNN trên train data đã được chuẩn hóa: 
knn_scaled = knn1.fit(X_train_scaled, y_train)

Để thực hiện dự báo chúng ta sử dụng mô hình đối với dữ liệu test data chuẩn hóa chứ không phải nguyên bản: 

In [4]:
# Chuẩn hóa test data: 
X_test_scaled = scaler.transform(X_test)

# Thực hiện dự báo trên test data đã chuẩn hóa: 
accuracy_scaled = knn_scaled.score(X_test_scaled, y_test)

# Accuracy của mô hình KNN: 
print("Accuracy nếu sử dụng dữ liệu chuẩn hóa:", accuracy_scaled)

Accuracy nếu sử dụng dữ liệu chuẩn hóa: 0.6355


Như vậy chỉ với kĩ thuật FE đơn giản là chuẩn hóa 0 - 1 thì chúng ta đã có thể cải thiện chất lượng dự báo của mô hình lên khoảng 10.32%: 

In [5]:
100*(accuracy_scaled / accuracy_noneScaled - 1)

10.329861111111116

Như vậy với chỉ cùng một giá trị của tham số K chúng ta có thể nâng cao chất lượng phân loại - dự báo của mô hình lên đáng kể. Nhưng trong thực tế, FE chỉ mới là một phần của quá trình tìm kiếm - xây dựng một mô hình ML đủ tốt/tốt hơn nữa. Chúng ta phải kết hợp FE với quá trình tinh chỉnh tham số cho mô hình như mục 4.2 dưới đây. 

# 4.2 Feature Engineering kết hợp Grid Search 

Tìm kiếm tham số tối ưu thông qua quá trình tinh chỉnh tham số mô hình là bước quan trọng của việc xây dựng mô hình ML trước khi nghĩ đến việc triển khai và ứng dụng nó. Chúng ta có thể kết hợp FE và tinh chỉnh tham số cho mô hình bằng Grid Search. Dưới đây là Python Codes cho tinh chỉnh tham số của mô hình SVM kết hợp với FE: 

In [6]:
# Load hàm SVC + GridSearchCV từ thư viện Scikit-Learn: 
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Thiết lập Grid Search cho tinh chỉnh: 
param_grid = {"C": [0.001, 0.01, 0.1, 1, 10, 100], "gamma": [0.001, 0.01, 0.1, 1, 10, 100]}

# Tinh chỉnh mô hình với 5 Fold Cross-Validation và huấn luyện luôn trên train đã được chuẩn hóa 0 - 1: 
grid = GridSearchCV(SVC(), param_grid, cv=5)
grid.fit(X_train_scaled, y_train)

# Accuracy trung bình cao nhất (cho 5 lần chạy mẫu) trên train data chuẩn hóa: 
best_Accuracy_fromCrossValidation = grid.best_score_

# Accuracy trên test data đã được chuẩn hóa: 
Accuracy_onTestData = grid.score(X_test_scaled, y_test)

# Hiển thị các kết quả: 
print("Accuracy trung bình cao nhất từ Cross-Validation:", best_Accuracy_fromCrossValidation)
print("Accuracy trên test data (đã chuẩn hóa): ", Accuracy_onTestData)

Accuracy trung bình cao nhất từ Cross-Validation: 0.747
Accuracy trên test data (đã chuẩn hóa):  0.7415


Chúng ta có thể chỉ ra tham số tối ưu mà khi huấn luyện SVM thì có Accuracy = 0.745 trên test data chuẩn hóa: 

In [7]:
print("Parameters tối ưu: ", grid.best_params_)

Parameters tối ưu:  {'C': 100, 'gamma': 0.1}


Đương nhiên kết quả Accuracy = 0.7415 có thể tìm lại bằng cách huyến luyện trực tiếp KNN với cặp tham số này rồi tính toán lại Accuracy cho test data chuẩn hóa: 

In [8]:
# Huấn luyện lại SVM với cặp tham số tối ưu gamma + C: 
svm_bestParameters = SVC(**grid.best_params_)

# Huấn luyện SVM và tính Accuracy: 
acc_bestParameters = svm_bestParameters.fit(X_train_scaled, y_train).score(X_test_scaled, y_test)

# Hiển thik kết quả: 
print("Accuracy trên test data (đã chuẩn hóa): ", acc_bestParameters)

Accuracy trên test data (đã chuẩn hóa):  0.7415


# 4.3 FE, Grid Search và Pipelines

Trong thực tế thì chúng ta có thể thực hiện một chuỗi các kĩ thuật biến đổi - chuyển hóa features nguyên bản thành features mới chứ không chỉ thực hiện chuẩn hóa 0-1 như đã thấy. Mặt khác, như đã thấy để xây dựng một mô hình ML với chuẩn hóa 0-1 chúng ta cần có thể phải chuẩn bị trước train và test data đã được chuẩn hóa trước khi xây dựng và đánh giá mô hình. Rất may mắn là Scikit-Learn có thể hỗ trợ cho khâu FE, huấn luyện và đánh giá mô hình theo một cách thức trình bày ngắn gọn và tiện lợi hơn rất nhiều bằng một "thủ tục" gọi là *Pipelines* (tôi chưa biết dịch là gì dù rằng nó trùng tên với bản hit Pipeline của ban nhạc The Ventures và tôi đã nghe từ lâu). Lợi ích của Pipelines là: 

- Gói gọn quá trình từ FE, tinh chỉnh, huấn luyện và đánh giá bằng một quy trình kín từ A-Z. 
- Trình bày ngắn gọn, tiện lợi cho việc huấn luyện, so sánh - lựa chọn nhiều mô hình ML khác nhau. 

Để minh họa thủ tục Pipelines chúng ta trở lại với KNN khi K = 1: 

In [9]:
# Load hàm Pipeline: 
from sklearn.pipeline import Pipeline

# Thiết lập thông số cho huấn luyện và chuẩn hóa 0 - 1: 
pipe_knn = Pipeline([("scaler", MinMaxScaler()), ("knn", KNeighborsClassifier(n_neighbors=1))])

# Huấn luyện KNN: 
pipe_knn.fit(X_train, y_train)

# Tính Accuracy: 
acc = pipe_knn.score(X_test, y_test)

# In Accuracy: 
print("Accuracy với FE là chuẩn hóa 0-1 cho test data:", acc)

Accuracy với FE là chuẩn hóa 0-1 cho test data: 0.6355


Sử dụng Pipelines cho FE + tinh chỉnh tham số theo cách tiếp cận Grid Search được thực hiện như Python Codes dưới đây. Để minh họa thì hai mô hình ML được chọn là KNN và SVM:  

In [10]:
#==============================================
# Pipelines cho FE + tinh chỉnh tham số, KNN
#==============================================

# Thiết lập thông số cho huấn luyện và chuẩn hóa 0 - 1: 
pipe_knn = Pipeline([("scaler", MinMaxScaler()), ("KNN", KNeighborsClassifier())])

# Định nghĩa các ứng viên tiềm năng cho tham số: 
param_grid_knn = {"KNN__n_neighbors": [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31]}

# Thiết lập môi trường tinh chỉnh: 
grid_search_knn = GridSearchCV(pipe_knn, param_grid_knn, cv=5)
grid_search_knn.fit(X_train, y_train)
print("Accuracy cao nhất từ Cross-Validation:", grid_search_knn.best_score_)
print("Accuracy cho Test Data: ", grid_search_knn.score(X_test, y_test))
print("Parameters tốt nhất: ", grid_search_knn.best_params_)

Accuracy cao nhất từ Cross-Validation: 0.7413333333333333
Accuracy cho Test Data:  0.741
Parameters tốt nhất:  {'KNN__n_neighbors': 29}


Một lần nữa giá trị Accuracy = 0.741 có thể được tính toán một cách trực tiếp như sau: 

In [11]:
# Huấn luyện lại KNN với tham số tối ưu tìm được: 
best_knn = KNeighborsClassifier(n_neighbors=29)

# Huấn luyện và tính Accuracy: 
acc_best_knn = best_knn.fit(X_train_scaled, y_train).score(X_test_scaled, y_test)

# In giá trị Accuracy: 
print("Accuracy tương ứng với tham số tối ưu, KNN:", acc_best_knn)

Accuracy tương ứng với tham số tối ưu, KNN: 0.741


Tương tự là sử dụng thủ tục Pipelines cho SVM - một mô hình có tham số tinh chỉnh là gamma và C: 

In [12]:
#==============================================
# Pipelines cho FE + tinh chỉnh tham số, SVM
#==============================================

# Chỉ thị cụ thể kĩ thuật FE và các ứng viên tiềm năng cho tham số: 
pipe_svm = Pipeline([("scaler", MinMaxScaler()), ("SVM", SVC())])
param_grid_svm = {"SVM__C": [10, 100], "SVM__gamma": [0.01, 0.1]}

# Load GridSearchCV: 
# from sklearn.model_selection import GridSearchCV

# Tinh chỉnh (tìm tham số tối ưu) và huấn luyện: 
grid_search_svm = GridSearchCV(pipe_svm, param_grid_svm, cv=5)
grid_search_svm.fit(X_train, y_train)

# Kết quả Accuracy trên test data và một số kết quả khác: 
print("Accuracy cao nhất từ Cross-Validation:", grid_search_svm.best_score_)
print("Accuracy cho Test Data: ", grid_search_svm.score(X_test, y_test))
print("Parameters tốt nhất: ", grid_search_svm.best_params_)

Accuracy cao nhất từ Cross-Validation: 0.747
Accuracy cho Test Data:  0.7415
Parameters tốt nhất:  {'SVM__C': 100, 'SVM__gamma': 0.1}


# 4.4 Tóm tắt chương

Chương này chỉ ra tác động của FE lên chất lượng phân loại - dự báo của các mô hình ML và sử dụng FE như là một "kĩ thuật" nâng cao chất lượng phân loại - dự báo. Để minh họa thì kĩ thuật chuẩn hóa 0-1 được lựa chọn với các mô hình KNN và SVM. Chương này cũng trình bày khía cạnh thực hành của việc kết hợp tinh chỉnh tham số theo cách tiếp cận Grid Search và FE. Chúng ta có thể áp dụng FE hoàn toàn tương tự cho: (1) các mô hình ML khác, (2) các kĩ thuật FE khác, và (3) kết hợp nhiều kĩ thuật FE cho cùng một thuật toán ML. 

Còn nhiều khía cạnh tinh vi hơn của FE áp dụng cho các mô hình ML nhưng trong phạm vi là một textbook cơ bản thì những điều này khó có thể trình bày trong chương. Bạn đọc quan tâm có thể tìm đến các tài liệu khác về chủ đề này. 

# Tài liệu tham khảo

1. James, G., Witten, D., Hastie, T., & Tibshirani, R. (2013). An introduction to statistical learning. New York: Springer.
2. Kuhn, M., & Johnson, K. (2013). Applied predictive modeling. New York: Springer.
3. Müller, A. C., & Guido, S. (2016). Introduction to machine learning with Python: a guide for data scientists. O'Reilly Media, Inc.
4. Géron, A. (2019). Hands-on machine learning with Scikit-Learn and TensorFlow: Concepts, tools, and techniques to build intelligent systems. O'Reilly Media, Inc.
5. Shmueli, G., Bruce, P. C., Yahav, I., Patel, N. R., & Lichtendahl Jr, K. C. (2017). Data mining for business analytics: concepts, techniques, and applications in R. John Wiley & Sons. 